### Accuracy evaluation
This notebook calculates proportions and creates the stratified random test set for the final level 2 land use cover maps for Laos and Vietnam. It also creates creates confusion matrices for the test set. Manual labelling of test samples was done outside of this notebook.

In [1]:
import concurrent.futures
import dask.array as da
import geopandas as gpd
import numpy as np
import os
import pandas as pd
import rasterio
import rioxarray
import shutil

from dask.diagnostics import ProgressBar
from PIL import Image
from rasterio.transform import rowcol, xy
from rasterio.windows import Window
from sklearn.metrics import confusion_matrix

from config import Config
from core.utils import get_mapping_from_csv

In [2]:
def calculate_pixel_proportions(raster_path, chunk_size=(1, 2048, 2048)):
    """Calculate pixel counts and proportions for a raster file."""
    # Open the raster with Dask for chunked processing
    raster = rioxarray.open_rasterio(raster_path, chunks=chunk_size)
    
    # Extract the first band (assuming a single-band raster)
    data = raster.isel(band=0)
    
    # Mask nodata values
    if raster.rio.nodata is not None:
        data = data.where(data != raster.rio.nodata)
    
    # # Flatten the array
    # flat_data = data.data.flatten()
    
    # Count unique values and their occurrences
    unique_values, counts = da.unique(data.data, return_counts=True)
    
    # Compute the results using Dask
    with ProgressBar():  # Show progress for Dask computation
        unique_values, counts = da.compute(unique_values, counts)
    
    # Calculate the total number of valid pixels
    total_pixels = counts.sum()
    
    # Convert the results to a Pandas DataFrame
    df = pd.DataFrame({
        "class": unique_values,
        "pixels": counts,
        "proportion": np.round(counts/total_pixels * 100, 4)
    })
    
    return df.dropna()

In [3]:
def get_window_samples_parallel(raster_path, window, luc_classes, samples_per_class, seed):
    """Get samples for a single window of the raster and maintain best samples."""
    rng = np.random.RandomState(seed)

    # Read the window
    with rasterio.open(raster_path) as src:
        data = src.read(1, window=window)

    # Get window offset for correct coordinates
    row_off, col_off = window.row_off, window.col_off

    samples = []
    for luc in luc_classes:
        # Find pixel locations with this class
        y_idxs, x_idxs = np.where(data == luc)
        
        if len(y_idxs) == 0:
            continue
            
        # Generate random values for each pixel of this class
        random_values = rng.random(len(y_idxs))  # Use the seeded RNG

        # Create array of samples with their random values
        # Add window offset to get global coordinates
        samples_class = np.column_stack([
            x_idxs + col_off,  # global x
            y_idxs + row_off,  # global y
            random_values,     # random value for later selection
            np.full_like(x_idxs, luc)  # class value
            ])
        samples.append(samples_class)
    
    if len(samples) == 0:
        return None
    
    df_samples = pd.DataFrame(np.concatenate(samples), columns=["x", "y", "random", "class"])
    idx_sample = df_samples.groupby("class")["random"].nsmallest(samples_per_class).reset_index()["level_1"]
    df_selected = df_samples.iloc[idx_sample].reset_index(drop=True)

    return df_selected

def get_stratified_random_samples_parallel(raster_path, luc_classes, samples_per_class, window_size=4096, seed=1234):
    """Get stratified random samples from a large raster using windows."""
    # Open raster file
    with rasterio.open(raster_path) as src:
        transform = src.transform
        height = src.height
        width = src.width
        
    # Process each window in parallel
    with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
        futures = []
        # Process raster in windows
        for i, (y, x) in enumerate([(y, x) for y in range(0, height, window_size) for x in range(0, width, window_size)]):
            # Define window ensuring we don't go out of bounds
            win_width = min(window_size, width - x)
            win_height = min(window_size, height - y)
            window = Window(x, y, win_width, win_height)
            window_seed = seed + i

            futures.append(executor.submit(
                get_window_samples_parallel,
                raster_path,
                window,
                luc_classes,
                samples_per_class,
                window_seed
                ))

    window_samples = [future.result() for future in futures if future.result() is not None]      
      
    # Combine samples for all windows and select final samples per class beased on lowest random value
    df_all_windows = pd.concat(window_samples).reset_index(drop=True)
    idx_final_sample = df_all_windows.groupby("class")["random"].nsmallest(samples_per_class).reset_index()["level_1"]
    df_final_sample = df_all_windows.iloc[idx_final_sample].sort_values(["class", "random"]).reset_index(drop=True)
    df_final_sample[["lon", "lat"]] = df_final_sample[["x", "y"]].apply(lambda x: xy(transform, x["y"], x["x"]), axis=1, result_type="expand")

    return df_final_sample


In [4]:
def process_raster_and_generate_patches(
    raster_path, 
    output_folder, 
    samples_sel, 
    patch_size=256, 
    center_size=9, 
    outer_alpha=100
):
    """
    Processes a raster file to generate image patches and saves them as PNG files.

    Args:
        raster_path (str): Path to the input raster file.
        output_folder (str): Directory where the generated patches will be saved.
        samples_sel (gpd.GeoDataFrame): GeoDataFrame containing sample coordinates in 'lon', 'lat' columns.
        patch_size (int, optional): Size of the patches to extract. Defaults to 255.
        center_size (int, optional): Size of the center area with full alpha. Defaults to 9.
        outer_alpha (int, optional): Alpha value for non-center pixels (0-255). Defaults to 100.

    Returns:
        None
    """
    # Ensure the output folder exists
    if os.path.exists(output_folder):
        shutil.rmtree(output_folder)
    os.makedirs(output_folder)

    # Calculate half size for patch extraction
    half_size = patch_size // 2

    # Open the raster file
    with rasterio.open(raster_path) as src:
        for i, row in samples_sel.iterrows():
            # Convert projected lat/lon to raster indices
            y, x = rowcol(src.transform, row["lon"], row["lat"])
            y, x = int(y), int(x)  # Ensure integer indices

            # Define patch boundaries
            x_min, x_max = x - half_size, x + half_size
            y_min, y_max = y - half_size, y + half_size

            # Handle edge cases (ensure we don't go out of bounds)
            x_min = max(0, x_min)
            x_max = min(src.width, x_max)
            y_min = max(0, y_min)
            y_max = min(src.height, y_max)

            # Read all bands (assuming it's a 3-band image)
            patch = src.read(window=((y_min, y_max), (x_min, x_max)))

            # Transpose to (Height, Width, Bands) format for image processing
            patch = np.transpose(patch, (1, 2, 0))
            if patch.shape != (patch_size, patch_size, 3):
                print(f"Skipping patch {i} due to shape mismatch: {patch.shape}")
                continue

            # Add alpha channel
            rgba_patch = np.dstack((patch, np.full((patch.shape[0], patch.shape[1]), 255, dtype=np.uint8)))

            # Determine the center coordinates
            center_start = (patch_size - center_size) // 2
            center_end = center_start + center_size

            # Set alpha for non-center pixels
            rgba_patch_alpha = rgba_patch[..., 3]
            rgba_patch_alpha[:, :] = outer_alpha
            rgba_patch_alpha[center_start:center_end, center_start:center_end] = 255

            # Convert to image and save as PNG
            img = Image.fromarray(rgba_patch.astype(np.uint8), mode="RGBA")

            # Extract the center area adding some padding
            center_patch = rgba_patch[
                center_start - center_size:center_end + center_size,
                center_start - center_size:center_end + center_size
            ]

            # Resize the center patch to the same size as the original patch
            center_patch_img = Image.fromarray(center_patch.astype(np.uint8), mode="RGBA")
            center_patch_img = center_patch_img.resize((patch_size, patch_size), Image.NEAREST)

            # Concatenate the original patch and the zoomed-in center patch
            combined_img = Image.new('RGBA', (patch_size * 2, patch_size))
            combined_img.paste(img, (0, 0))
            combined_img.paste(center_patch_img, (patch_size, 0))

            # Save the combined image
            img_path = f"{output_folder}/patch_{i}.png"
            combined_img.save(img_path)

In [5]:
def extract_raster_values_to_gdf(gdf, raster_path, new_col='raster_value'):
    """
    Extract raster values at the locations of a GeoDataFrame and add them as a new column.

    Parameters:
        gdf (gpd.GeoDataFrame): GeoDataFrame with point geometries.
        raster_path (str): Path to the raster file.
        new_col (str): Name of the new column to store raster values.

    Returns:
        gpd.GeoDataFrame: GeoDataFrame with the new column containing raster values.
    """
    # Ensure the GeoDataFrame has point geometries
    if not all(gdf.geometry.geom_type == 'Point'):
        raise ValueError("GeoDataFrame must contain Point geometries.")

    # Open the raster file
    with rasterio.open(raster_path) as src:
        # Extract coordinates from the GeoDataFrame
        coords = [(geom.x, geom.y) for geom in gdf.geometry]
        
        # Sample the raster at the specified coordinates
        values = [val[0] for val in src.sample(coords)]
    
    # Add the values as a new column to the GeoDataFrame
    gdf[new_col] = values
    return gdf

In [6]:
config = Config.Config()

In [7]:
os.makedirs(config.acc_eval_folder, exist_ok=True)

# Calculate class proportions

In [8]:
class_mapping_lao = get_mapping_from_csv(config.topo_legend_path, col_key="pixel_post2", col_value="class_l2")
class_mapping_vnm = get_mapping_from_csv(config.topo_legend_path, col_key="pixel_post3", col_value="class_l2_vnm")

In [9]:
# Evaluation raster class counts - Lao
prop = calculate_pixel_proportions(config.topo_eval_lao)
prop["class_name"] = prop["class"].map(class_mapping_lao)
prop[["class", "class_name", "pixels", "proportion"]].to_csv(config.class_counts_eval_lao, index=False)

[########################################] | 100% Completed | 204.39 s


In [10]:
# Stratum counts based on initial results used for stratified sampling of test set
prop = calculate_pixel_proportions(config.topo_sampling_lao)
prop["class_name"] = prop["class"].map(class_mapping_lao)
prop[["class", "class_name", "pixels", "proportion"]].to_csv(config.strata_counts_eval_lao, index=False)

[########################################] | 100% Completed | 211.67 s


In [11]:
# Evaluation raster class counts - Vietnam
prop = calculate_pixel_proportions(config.topo_eval_vnm)
prop["class_name"] = prop["class"].map(class_mapping_vnm)
prop[["class", "class_name", "pixels", "proportion"]].to_csv(config.class_counts_eval_vnm, index=False)

[########################################] | 100% Completed | 375.48 s


In [12]:
# Stratum counts based on initial results used for stratified sampling of test set
prop = calculate_pixel_proportions(config.topo_sampling_vnm)
prop["class_name"] = prop["class"].map(class_mapping_vnm)
prop[["class", "class_name", "pixels", "proportion"]].to_csv(config.strata_counts_eval_vnm, index=False)

[########################################] | 100% Completed | 398.53 s


# Test data - Lao

In [13]:
luc_classes = list(range(1, config.classes))

In [14]:
samples = get_stratified_random_samples_parallel(
    config.topo_sampling_lao,
    luc_classes=luc_classes,
    samples_per_class=config.n_test_samples_init_per_class,
    seed=config.seed
    )
samples.to_csv(config.test_samples_init_lao, index=False)

In [15]:
prop = pd.read_csv(config.strata_counts_eval_lao)
samples = pd.read_csv(config.test_samples_init_lao)

# Select specific number of test samples per class and the rest by proportion
sample_num = np.round(config.n_test_samples_sel_by_prop * prop["proportion"]/100 + config.n_test_samples_sel_per_class).astype(int)
sample_num.index = prop["class"].astype("int")

# Create an empty DataFrame to store the selected rows
samples_sel = pd.DataFrame()
# Loop through each class index and select the first n rows
for class_index, num_rows in sample_num.items():
    class_rows = samples[samples["class"] == class_index].head(num_rows)
    samples_sel = pd.concat([samples_sel, class_rows])

# Randomly shuffle
samples_sel = samples_sel.sample(frac=1, random_state=config.seed).reset_index(drop=True)
samples_sel.to_csv(config.test_samples_lao, index=False)

# Convert to GeoDataFrame
samples_sel = gpd.GeoDataFrame(
    samples_sel, geometry=gpd.points_from_xy(samples_sel.lon, samples_sel.lat), crs=config.output_crs
)
samples_sel.to_file(config.test_samples_geo_lao)

# Write out sample patches for manual labelling
process_raster_and_generate_patches(config.map_sheets_merged_path, config.test_samples_folder_lao, samples_sel)

samples_sel.head(3)

,x,y,random,class,lon,lat,geometry
0,68223.0,109356.0,1.679006e-04,1.0,-2.601397e+06,3.608490e+06,POINT (-2601396.535 3608489.953)
1,45141.0,62870.0,1.633013e-08,6.0,-2.693725e+06,3.794434e+06,POINT (-2693724.535 3794433.953)
2,41672.0,55508.0,1.507515e-08,6.0,-2.707601e+06,3.823882e+06,POINT (-2707600.535 3823881.953)


# Test data - Vietnam

In [16]:
luc_classes = list(range(1, config.classes))

In [17]:
samples = get_stratified_random_samples_parallel(
    config.topo_sampling_vnm,
    luc_classes=luc_classes,
    samples_per_class=config.n_test_samples_init_per_class,
    seed=config.seed
    )
samples.to_csv(config.test_samples_init_vnm, index=False)

In [18]:
prop = pd.read_csv(config.strata_counts_eval_vnm)
samples = pd.read_csv(config.test_samples_init_vnm)

# Select specific number of test samples per class and the rest by proportion
sample_num = np.round(config.n_test_samples_sel_by_prop * prop["proportion"]/100 + config.n_test_samples_sel_per_class).astype(int)
sample_num.index = prop["class"].astype("int")

# Create an empty DataFrame to store the selected rows
samples_sel = pd.DataFrame()
# Loop through each class index and select the first n rows
for class_index, num_rows in sample_num.items():
    class_rows = samples[samples["class"] == class_index].head(num_rows)
    samples_sel = pd.concat([samples_sel, class_rows])

# Randomly shuffle
samples_sel = samples_sel.sample(frac=1, random_state=config.seed).reset_index(drop=True)
samples_sel.to_csv(config.test_samples_vnm, index=False)

# Convert to GeoDataFrame
samples_sel = gpd.GeoDataFrame(
    samples_sel, geometry=gpd.points_from_xy(samples_sel.lon, samples_sel.lat), crs=config.output_crs
)
samples_sel.to_file(config.test_samples_geo_vnm)

# Write out sample patches for manual labelling
process_raster_and_generate_patches(config.map_sheets_merged_path, config.test_samples_folder_vnm, samples_sel)

samples_sel.head(3)

,x,y,random,class,lon,lat,geometry
0,150885.0,321558.0,8.735614e-06,1.0,-2.050409e+06,2.878074e+06,POINT (-2050408.535 2878073.953)
1,86584.0,51708.0,6.840070e-09,6.0,-2.307613e+06,3.957474e+06,POINT (-2307612.535 3957473.953)
2,141652.0,47719.0,5.799617e-09,6.0,-2.087341e+06,3.973430e+06,POINT (-2087340.535 3973429.953)


# Labels and confusion matrix
Labels were created manually outside of this notebook (based on the selected pixels above) and are read in below to calculate the corresponding model predictions and confusion matrices.

### Lao

In [19]:
# Merge test predictions to labels
labels = pd.read_csv(config.test_labels_lao)
labels = gpd.GeoDataFrame(
    labels, geometry=gpd.points_from_xy(labels.lon, labels.lat), crs=config.output_crs
)
labels = extract_raster_values_to_gdf(labels, config.topo_eval_lao, new_col="pred")
labels = labels.rename({"class": "stratum"}, axis=1)
labels.drop("geometry", axis=1).to_csv(config.test_labels_pred_lao, index=False)

# Confusion matrix
counts = pd.read_csv(config.strata_counts_eval_lao)
colnames = counts["class_name"].str.capitalize().values
confm = confusion_matrix(labels["label"], labels["pred"])
confm = pd.DataFrame(confm, columns=colnames, index=colnames)
confm["Total"] = confm.sum(axis=1)
confm.loc["Total"] = confm.sum(axis=0)
confm.to_csv(config.confusion_matrix_lao)
confm

,Built-up,Rice,Forest,Brushwood,Plantation,Swamp,Inundation,Water,Grassland or bare,Sand,Total
Built-up,79,0,0,0,0,0,0,1,0,0,80
Rice,0,105,1,0,0,0,0,0,1,3,110
Forest,1,2,516,3,0,5,0,2,1,1,531
Brushwood,0,0,2,167,0,0,0,1,0,0,170
Plantation,0,0,0,1,95,0,0,0,1,0,97
Swamp,0,2,1,0,0,90,2,0,0,0,95
Inundation,0,0,1,0,0,1,98,0,0,0,100
Water,0,0,2,0,0,0,0,93,3,0,98
Grassland or bare,4,3,5,3,0,1,0,6,105,11,138
Sand,0,0,1,1,0,0,0,0,8,71,81


### Vietnam

In [20]:
# Merge test predictions to labels
labels = pd.read_csv(config.test_labels_vnm)
labels = gpd.GeoDataFrame(
    labels, geometry=gpd.points_from_xy(labels.lon, labels.lat), crs=config.output_crs
)
labels = extract_raster_values_to_gdf(labels, config.topo_eval_vnm, new_col="pred")
labels = labels.rename({"class": "stratum"}, axis=1)
labels.drop("geometry", axis=1).to_csv(config.test_labels_pred_vnm, index=False)

# Confusion matrix
counts = pd.read_csv(config.strata_counts_eval_vnm)
colnames = counts["class_name"].str.capitalize().values
confm = confusion_matrix(labels["label"], labels["pred"])
confm = pd.DataFrame(confm, columns=colnames, index=colnames)
confm["Total"] = confm.sum(axis=1)
confm.loc["Total"] = confm.sum(axis=0)
confm.to_csv(config.confusion_matrix_vnm)
confm

,Built-up,Rice,Forest or brushwood,Plantation,Swamp,Inundation,Mangrove,Water,Grassland or bare,Sand,Total
Built-up,87,0,0,0,0,0,0,0,7,0,94
Rice,0,179,0,0,2,0,0,0,3,0,184
Forest or brushwood,0,0,443,1,0,0,0,1,1,0,446
Plantation,0,2,0,99,0,0,0,1,0,0,102
Swamp,0,1,0,0,105,0,1,0,2,0,109
Inundation,0,0,0,0,1,103,0,0,0,0,104
Mangrove,0,0,0,0,1,0,105,0,1,0,107
Water,0,1,1,0,0,0,0,107,1,2,112
Grassland or bare,3,7,5,0,0,0,0,1,137,0,153
Sand,0,0,0,0,0,0,0,1,2,86,89
